In [1]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators
import base64, pickle

In [2]:
%ls ../examples/data/

9PC1X-complex.oeb.gz*     receptor-fixed.pdb*    toluene.pdb*
9PC1X-simulation.oeb.gz*  smirff99Frosst.ffxml*  WNE6S-minimized.oeb.gz
9PC1X-smirff.oeb.gz*      T4-protein.pdb*        WNE6S-warmup.oeb.gz
IssueGetVelocities.ipynb  test-receptor.oeb.gz*
mcmol.oeb*                test_smiles.ism*


In [3]:
# Check tagged data in minimized OEB
mol = oechem.OEMol()
with oechem.oemolistream('../examples/data/WNE6S-minimized.oeb.gz') as ifs:
    if not oechem.OEReadMolecule(ifs, mol):
        raise RuntimeError("Error reading complex")
print(mol.GetData().keys())

dict_keys(['OpenMM_PLmaskDict_json', 'state', 'structure', 'system', 'idtag'])


In [4]:
# Regenerate System/Structure from tagged data
sys_tag = oechem.OEGetTag('system')
encoded_system = mol.GetData(sys_tag)
system = openmm.XmlSerializer.deserialize(encoded_system)
print(type(system))
struct_tag = oechem.OEGetTag('structure')
encoded_structure = mol.GetData(struct_tag)
decoded_structure = base64.b64decode(encoded_structure)
struct_dict = pickle.loads(decoded_structure)
struct = parmed.structure.Structure()
struct.__setstate__(struct_dict)
positions = struct.positions
topology = struct.topology
print(topology)

<class 'simtk.openmm.openmm.System'>
<Topology; 3 chains, 10097 residues, 32357 atoms, 22445 bonds>


In [5]:
# Grab the saved state, check for velocities
encoded_state = mol.GetData(oechem.OEGetTag('state'))
saved_state = openmm.XmlSerializer.deserialize(encoded_state)
saved_vel = saved_state.getVelocities()
not np.any(saved_vel)

TypeError: Velocities were not requested in getState() call, so are not available.

Looks like something went wrong in trying to store the velocities in the saved state??? Check for positions...

In [6]:
not np.any(saved_state.getPositions())

False

Positions look to be stored okay...Let's try running a short simulation and saving the velocities again

In [7]:
saved_positions = saved_state.getPositions()

In [8]:
# Now regenerate the simulation object.
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picoseconds, 0.002*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator)

In [9]:
sim_state = simulation.context.getState(getPositions=True,getVelocities=True,getParameters=True)
sim_vel = sim_state.getVelocities(asNumpy=True) 
not np.any(sim_vel)

True

In [10]:
simulation.context.setPositions(saved_positions)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

Set the positions to the saved positions and reset the velocities. Then check the velocities is no longer None

In [11]:
sim_state = simulation.context.getState(getPositions=True,getVelocities=True,getParameters=True)
sim_vel = sim_state.getVelocities(asNumpy=True) 
not np.any(sim_vel)

False

In [12]:
simulation.step(5000)

In [13]:
state = simulation.context.getState( getPositions=True,
                                     getVelocities=True,
                                     getForces=True,
                                     getEnergy=True,
                                     getParameters=True,
                                     enforcePeriodicBox=True )
state.getVelocities()[0]

Quantity(value=(-0.21839779615402222, -0.3398388624191284, -0.8523949980735779), unit=nanometer/picosecond)

Encode the system and newly simulated state and write out to another OEB file

In [14]:
outmol = oechem.OEMol()
ofs = oechem.oemolostream('WNE6S-simulation.oeb.gz')
encoded_system = openmm.XmlSerializer.serialize(system).encode()
encoded_state = openmm.XmlSerializer.serialize(state).encode()
outmol.SetData(oechem.OEGetTag('system'), encoded_system)
outmol.AddData(oechem.OEGetTag('state'), encoded_state)
oechem.OEWriteConstMolecule(ofs, outmol)
ofs.close()

Read in the simulated OEB file and check for the newly saved state

In [15]:
simmol = oechem.OEMol()
with oechem.oemolistream('WNE6S-simulation.oeb.gz') as ifs:
    oechem.OEReadMolecule(ifs,simmol)

In [16]:
saved_system = openmm.XmlSerializer.deserialize( simmol.GetData(oechem.OEGetTag('system')) )
saved_state = openmm.XmlSerializer.deserialize( simmol.GetData(oechem.OEGetTag('state')) )

In [17]:
saved_state.getVelocities()[0]

Quantity(value=(-0.21839779615402222, -0.3398388624191284, -0.8523949980735779), unit=nanometer/picosecond)

Looks like the velocities were preserved in this case?